## Transfer Learning Models

## Import libraries

In [ ]:
import tensorflow as tf 
from tensorflow.keras.applications import MobileNetV2, ResNet50, EfficientNetB0,InceptionV3,VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import os

## Declare Constants

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50
input_shape = (BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)

## Load Train Dataset

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../data/train",
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

## Load Test Datset

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../data/test",
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
)

## Load Validation Dataset

In [ ]:
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../data/val",
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
)

In [ ]:
class_names = train_dataset.class_names
n_classes = len(class_names)

In [ ]:
class_names

## Prepare Datasets for Performance

In [ ]:

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = valid_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

## Load PreTrained Model

### The build_model function:

- Loads a pre-trained CNN model (like MobileNet, ResNet, etc.) without the top classification layer (include_top=False).

- Freezes the base model’s weights (no training).

- Adds custom layers on top:

    - GlobalAveragePooling2D

    - Dropout (to reduce overfitting)

    - Dense softmax output layer for classification.

- Compiles the model with:

    - adam optimizer

    - sparse_categorical_crossentropy loss

    - accuracy metric

### Returns the final compiled model.

In [ ]:
IMAGE_SIZE = (224, 224)
def build_model(base_model_fn, model_name):
    print(f"\nBuilding model: {model_name}")
    base_model = base_model_fn(
        input_shape=IMAGE_SIZE + (3,),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # Freeze base model

    inputs = tf.keras.Input(shape=IMAGE_SIZE + (3,))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

## List of Pretrained Models to Try

In [ ]:

pretrained_models = {
    "MobileNetV2": MobileNetV2,
    "ResNet50": ResNet50,
    "EfficientNetB0": EfficientNetB0, 
    "InceptionV3" : InceptionV3,
    "VGG16" : VGG16
}

## Train Each Model and Collect Results

In [ ]:
histories = {}
val_accuracies = {}

for name, base_model_fn in pretrained_models.items():
    model = build_model(base_model_fn, name)
    
    early_stop = EarlyStopping(patience=3, restore_best_weights=True)
    
    print(f"Training {name}...")
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=50,
        callbacks=[early_stop],
        verbose=1
    )

    val_acc = max(history.history["val_accuracy"])
    val_accuracies[name] = val_acc
    histories[name] = history
    print(f"{name} max val_accuracy: {val_acc:.4f}")

## Get the best Model

In [ ]:

best_model_name = max(val_accuracies, key=val_accuracies.get)
print(f"\n Best model is: {best_model_name} with validation accuracy: {val_accuracies[best_model_name]:.4f}")

## Rebuild the Best Model (for test + fine-tuning)

In [ ]:
def build_model(base_model_fn, model_name, trainable=False):
    print(f"\nBuilding model: {model_name}")
    base_model = base_model_fn(
        input_shape=IMAGE_SIZE + (3,),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = trainable  # Control trainable param

    inputs = tf.keras.Input(shape=IMAGE_SIZE + (3,))
    x = base_model(inputs, training=not trainable)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

## Re-evaluate Best Model on Test Data

In [ ]:
# Rebuild best model (frozen)
best_model = build_model(pretrained_models[best_model_name], best_model_name, trainable=False)

# Train briefly (for consistent weights)
best_model.fit(train_dataset, validation_data=val_dataset, epochs=5)

# Evaluate on test set
test_loss, test_acc = best_model.evaluate(test_dataset)
print(f"Test Accuracy (frozen): {test_acc:.4f}")

## Fine-Tune the Best Model

In [ ]:
# Unfreeze top layers for fine-tuning
fine_tune_at = 100

base_model = best_model.layers[1]  # Get base model from sequential
base_model.trainable = True

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile with low learning rate for fine-tuning
best_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

# Fine-tune training
fine_tune_history = best_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5,
    callbacks=[EarlyStopping(patience=2, restore_best_weights=True)]
)

# Final test accuracy
test_loss_ft, test_acc_ft = best_model.evaluate(test_dataset)
print(f"Test Accuracy (fine-tuned): {test_acc_ft:.4f}")

## Compare Before & After Fine-Tuning

In [ ]:
print(f"Before fine-tuning test acc: {test_acc:.4f}")
print(f"After fine-tuning  test acc: {test_acc_ft:.4f}")

## Extract metrics from fine-tuning history

In [ ]:
acc = fine_tune_history.history['accuracy']
val_acc = fine_tune_history.history['val_accuracy']
loss = fine_tune_history.history['loss']
val_loss = fine_tune_history.history['val_loss']
epochs_ran = len(acc)

## Accuracy and Validation Plot

In [ ]:
plt.figure(figsize=(10, 5))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(range(epochs_ran), acc, label='Training Accuracy')
plt.plot(range(epochs_ran), val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend(loc='lower right')
plt.savefig('../results/metrices/TransferLearning/Training_vs_Validation_Accuracy.png')

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(range(epochs_ran), loss, label='Training Loss')
plt.plot(range(epochs_ran), val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend(loc='upper right')
plt.savefig('../results/metrices/TransferLearning/Training_vs_Validation_Loss.png')

plt.tight_layout()
plt.show()

## Making prediction on single image from test dataset

In [ ]:
import numpy as np
# Iterate through one batch from the test dataset
for images_batch, labels_batch in test_dataset.take(1):
    
    # Extract first image and label
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    # Show image
    print("First image to predict:")
    plt.imshow(first_image)
    plt.axis('off')  # Hide axis for cleaner display
    plt.show()

    print("Actual label:", class_names[first_label])

    # Make prediction using fine-tuned best model
    batch_prediction = best_model.predict(images_batch)
    predicted_class = class_names[np.argmax(batch_prediction[0])]

    print("Predicted label:", predicted_class)

## Make Predictions

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension

    predictions = model.predict(img_array, verbose=0)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * np.max(predictions[0]), 2)
    return predicted_class, confidence

## Display 9 images with predictions

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(best_model, images[i])
        actual_class = class_names[labels[i].numpy()]
        
        plt.title(f"Actual: {actual_class}\nPredicted: {predicted_class}\nConfidence: {confidence}%", fontsize=10)
        plt.axis("off")

plt.tight_layout()
plt.show()

# Performance Metrices

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
# 1. Evaluate the model
test_loss, test_accuracy = best_model.evaluate(test_dataset)
print("Test Accuracy:", round(test_accuracy * 100, 2), "%")

# 2. Collect predictions
y_true = []
y_pred = []

for images, labels in test_dataset:
    preds = best_model.predict(images, verbose=0)
    predicted_classes = np.argmax(preds, axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(predicted_classes)

# 3. Classification Report
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

# 4. Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()

## Collecting prediction probabilities from best_model on the entire test dataset

In [ ]:
y_pred_probs = []

for images, _ in test_dataset:
    probs = best_model.predict(images, verbose=0)
    y_pred_probs.extend(probs)

y_pred_probs = np.array(y_pred_probs)

## ROC-AUC Curve (Multiclass OvR)

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

n_classes = y_pred_probs.shape[1]
y_true_binarized = label_binarize(y_true, classes=list(range(n_classes)))

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_binarized[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure(figsize=(8, 6))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'{class_names[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curve')
plt.legend(loc='lower right')
plt.grid(True)

# Save the figure
plt.savefig("../results/metrices/TransferLearning/Transfer_Learning_roc_curve.png")
plt.show()

## Precission-Recall Curve

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
import numpy as np

# 1. Generate prediction probabilities from the fine-tuned best model
y_pred_probs = []

for images, _ in test_dataset:
    probs = best_model.predict(images, verbose=0)
    y_pred_probs.extend(probs)

y_pred_probs = np.array(y_pred_probs)

# 2. Binarize true labels
from sklearn.preprocessing import label_binarize

n_classes = y_pred_probs.shape[1]
y_true_binarized = label_binarize(y_true, classes=list(range(n_classes)))

# 3. Plot Precision-Recall Curve
plt.figure(figsize=(8, 6))

for i in range(n_classes):
    precision, recall, _ = precision_recall_curve(y_true_binarized[:, i], y_pred_probs[:, i])
    ap = average_precision_score(y_true_binarized[:, i], y_pred_probs[:, i])
    plt.plot(recall, precision, label=f'{class_names[i]} (AP = {ap:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Multiclass Precision-Recall Curve (Fine-Tuned Model)')
plt.legend(loc='lower left')
plt.grid(True)

# 4. Save the plot
plt.savefig("../results/metrices/TransferLearning/Transfer_Learning_precision_recall_curve.png")
plt.show()

## F1-score vs. Threshold

In [ ]:
from sklearn.metrics import f1_score

# Select the class index you want to analyze (e.g., 0 for the first class)
class_index = 0
class_name = class_names[class_index]  # Get readable class name

# Thresholds to evaluate
thresholds = np.linspace(0, 1, 100)
f1s = []

# Loop through thresholds to compute F1 scores
for thresh in thresholds:
    preds_thresh = (y_pred_probs[:, class_index] >= thresh).astype(int)
    f1 = f1_score(y_true_binarized[:, class_index], preds_thresh)
    f1s.append(f1)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(thresholds, f1s, color='purple')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.title(f'F1 Score vs Threshold ({class_name})')
plt.grid(True)

# Save and show the plot
plt.savefig(f"../results/metrices/TransferLearning/Transfer_Learning_CNN_f1_score_vs_threshold_{class_name}.png")
plt.show()

## Save Final Best Model

In [ ]:
best_model.save(f"../models/{best_model_name}_(Transfer_Learning)_fine_tuned_model.keras")
print(f"Model saved as: {best_model_name}_(Transfer_Learning)_fine_tuned_model.keras")